In [1]:
import logging
from utility import add_project_root_to_path

logging.basicConfig(level=logging.ERROR)

add_project_root_to_path()

In [2]:
# Building experiments configs

from fee_algorithm.fixed_fee import FixedFee
from fee_algorithm.discrete_fee_perfect_oracle import DiscreteFeePerfectOracle
from fee_algorithm.based_on_trade_count_fee import BasedOnTradeCountFee
from fee_algorithm.adaptive_fee_based_on_block_price_move import AdaptiveBasedOnPreviousBlockPriceMoveFee

from copy import deepcopy

# TODO: move to one place with historical data experiments
fee_algos_to_consider = {
    "FX_fee": FixedFee(exchange_fee_rate=0.003), # 30 bps
    "DA_fee": BasedOnTradeCountFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "BA_fee": AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "OB_fee": DiscreteFeePerfectOracle(fee_rate_in_arbitrage_direction=0.0045, fee_rate_in_non_arbitrage_direction=0.0015), # 45/15 bps
}

In [3]:
from experiments.experiment import SyntheticDataDescription, GBMParameters
from datetime import datetime
import numpy as np

# Provided data, see synthetic_data_example.ipynb

gbm_parameters_by_alias = {
    "volatile_market": GBMParameters(
        S0=[3406.04, 1.299e-05],
        mu=[2.6092527015913387e-07, 0.0003608078612459716],
        cov_matrix=np.array([
            [5.91363022e-07, 1.20049585e-06],
            [1.20049585e-06, 3.40028539e-05]
        ])
    ),
    "calm_market": GBMParameters(
        S0=[3179.96, 1.587e-05],
        mu=[-5.439664535383297e-06, -3.1348041846428764e-06],
        cov_matrix=np.array(
            [[1.62154349e-06, 1.31591401e-06],
            [1.31591401e-06, 1.92476086e-06]]
        )
    ),
    "bull_market": GBMParameters(
        S0=[2518.75, 1.751e-05],
        mu=[8.82932404942772e-06, 1.0120694501213374e-05],
        cov_matrix=np.array(
            [[1.04648385e-06, 1.37680192e-06],
            [1.37680192e-06, 4.61054708e-06]]
        )
    ),
    "bear_market": GBMParameters(
        S0=[3539.93, 2.858e-05],
        mu=[-3.5843786641630628e-06, -4.891135343153541e-06],
        cov_matrix=np.array(
            [[9.69680101e-07, 1.32425664e-06],
            [1.32425664e-06, 3.19313094e-06]]
        )
    )
}

# Time range is used only to estimate number of "blocks" in simulation, so all time ranges are exactly 24 hours
time_range_by_alias = {
    "volatile_market": (datetime(2024, 3, 1, 12, 0, 0), datetime(2024, 3, 2, 12, 0, 0)),
    "calm_market": (datetime(2024, 8, 1, 12, 0, 0), datetime(2024, 8, 2, 12, 0, 0)),
    "bull_market": (datetime(2024, 11, 1, 12, 0, 0), datetime(2024, 11, 2, 12, 0, 0)),
    "bear_market": (datetime(2024, 4, 1, 12, 0, 0), datetime(2024, 4, 2, 12, 0, 0)),
}

synthetic_data_description_by_alias = {
    alias: SyntheticDataDescription(
        gbm_parameters=gbm_parameters_by_alias[alias],
        start_time=time_range_by_alias[alias][0],
        end_time=time_range_by_alias[alias][1],
    ) for alias in gbm_parameters_by_alias.keys()
}

random_seeds = [i for i in range(100)]

In [4]:
from experiments.experiment import Experiment
from experiments.configs import DEFAULT_UNINFORMED_USERS_CONFIG
from user.informed_user import InformedUser

# build experiments config

config_by_fee_algo = {}

for fee_algo_name, fee_algo in fee_algos_to_consider.items():
    for data_alias, synthetic_data_description in synthetic_data_description_by_alias.items():
        config_by_fee_algo[f"{data_alias}|{fee_algo_name}"] = Experiment(
            data=synthetic_data_description, 
            fee_algorithm=deepcopy(fee_algo),
            informed_user=InformedUser(),
            uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
        )

In [5]:
from experiments.run_experiment import run_seeded_experiment

In [6]:
from tqdm import tqdm

results_by_fee_algo = {}

for fee_algo_name, experiment in tqdm(config_by_fee_algo.items()):
    results_by_fee_algo[fee_algo_name] = run_seeded_experiment(experiment, random_seeds=random_seeds)

100%|██████████| 16/16 [02:59<00:00, 11.21s/it]


In [7]:
all_experiments_results = {
    f"{fee_algo_name}|{random_seeds[i]}": results[i]
    for fee_algo_name, results in results_by_fee_algo.items()
    for i in range(len(results))
}

In [8]:
from visualizations.compare_fee_algoritms import get_experiment_summary

df = get_experiment_summary(
    all_experiments_results
)

In [9]:
df

,experiment_name,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,volatile_market|FX_fee|0,72088.04,706,-7605.94,743,-71727.10,86871.67
1,volatile_market|FX_fee|1,97402.37,759,-8678.48,708,-96058.89,2249964.03
2,volatile_market|FX_fee|2,88193.43,714,-9461.41,761,-86107.02,3621769.83
3,volatile_market|FX_fee|3,70844.16,721,-8113.68,739,-70030.48,-71726.87
4,volatile_market|FX_fee|4,80918.41,741,-9442.17,724,-78801.24,1413659.80
...,...,...,...,...,...,...,...
1595,bear_market|OB_fee|95,877.39,107,-8352.30,730,3289.90,-4175.69
1596,bear_market|OB_fee|96,616.26,88,-8283.55,736,3547.29,-5115.41
1597,bear_market|OB_fee|97,894.64,108,-7933.58,721,2893.94,26922.32
1598,bear_market|OB_fee|98,947.36,97,-8008.14,720,2975.78,-6034.52


In [10]:
# 1. Split on underscores
df['split'] = df['experiment_name'].str.split('|')

# 2. Create columns based on slices of the split parts
df['data_alias'] = df['split'].apply(lambda x: x[0])
df['fee_algo']   = df['split'].apply(lambda x: x[1])
df['seed']       = df['split'].apply(lambda x: x[2])

# 3. Drop the split column
df = df.drop(columns=['split'])

In [11]:
df

,experiment_name,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss,data_alias,fee_algo,seed
0,volatile_market|FX_fee|0,72088.04,706,-7605.94,743,-71727.10,86871.67,volatile_market,FX_fee,0
1,volatile_market|FX_fee|1,97402.37,759,-8678.48,708,-96058.89,2249964.03,volatile_market,FX_fee,1
2,volatile_market|FX_fee|2,88193.43,714,-9461.41,761,-86107.02,3621769.83,volatile_market,FX_fee,2
3,volatile_market|FX_fee|3,70844.16,721,-8113.68,739,-70030.48,-71726.87,volatile_market,FX_fee,3
4,volatile_market|FX_fee|4,80918.41,741,-9442.17,724,-78801.24,1413659.80,volatile_market,FX_fee,4
...,...,...,...,...,...,...,...,...,...,...
1595,bear_market|OB_fee|95,877.39,107,-8352.30,730,3289.90,-4175.69,bear_market,OB_fee,95
1596,bear_market|OB_fee|96,616.26,88,-8283.55,736,3547.29,-5115.41,bear_market,OB_fee,96
1597,bear_market|OB_fee|97,894.64,108,-7933.58,721,2893.94,26922.32,bear_market,OB_fee,97
1598,bear_market|OB_fee|98,947.36,97,-8008.14,720,2975.78,-6034.52,bear_market,OB_fee,98


In [12]:
import pandas as pd

def show_stats_for_data_alias(data_alias):
    df_filtered = df[df['data_alias'] == data_alias]


    grouped_stats = df_filtered.drop(columns=["experiment_name", "data_alias", "seed"]).groupby('fee_algo').agg(['mean', 'std'])

    formatted_stats = grouped_stats.apply(
        lambda row: [f"{row[col, 'mean']:.2f} ± {row[col, 'std']:.2f}" for col in grouped_stats.columns.levels[0]],
        axis=1
    )

    formatted_stats_df = pd.DataFrame(formatted_stats.tolist(), columns=grouped_stats.columns.levels[0], index=grouped_stats.index)

    formatted_stats_df.reset_index(inplace=True)

    display(formatted_stats_df)

In [13]:
# Volatile Market

show_stats_for_data_alias("volatile_market")

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,85316.89 ± 7955.15,732.03 ± 20.22,-8746.39 ± 593.86,719.63 ± 17.34,-83828.80 ± 7775.60,1372909.63 ± 1271359.29
1,DA_fee,86112.12 ± 8107.64,734.16 ± 20.64,-8663.79 ± 618.63,717.33 ± 18.14,-84705.78 ± 7875.79,1374960.63 ± 1272116.16
2,FX_fee,86139.64 ± 8108.12,734.36 ± 20.77,-8672.71 ± 613.62,717.33 ± 18.25,-84725.38 ± 7878.42,1374582.45 ± 1271978.42
3,OB_fee,70229.34 ± 6942.50,599.27 ± 19.82,-8879.65 ± 577.39,720.61 ± 19.35,-67949.09 ± 6758.05,1355404.49 ± 1270692.57


In [14]:
# Calm Market

show_stats_for_data_alias("calm_market")

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,594.73 ± 104.31,92.48 ± 11.44,-8093.43 ± 292.86,719.38 ± 18.02,3439.40 ± 227.32,-3501.33 ± 5781.78
1,DA_fee,594.27 ± 101.08,94.10 ± 12.54,-8106.37 ± 292.12,719.60 ± 17.22,3443.60 ± 219.95,-3464.84 ± 6058.49
2,FX_fee,585.67 ± 101.25,94.05 ± 12.02,-8072.99 ± 289.38,718.34 ± 17.45,3425.37 ± 224.74,-3447.69 ± 6002.89
3,OB_fee,422.22 ± 83.11,68.18 ± 11.00,-8105.40 ± 281.76,720.29 ± 18.23,3740.83 ± 232.94,-3758.74 ± 5959.20


In [15]:
# Bull Market

show_stats_for_data_alias("bull_market")

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,2899.09 ± 385.00,209.64 ± 17.74,-8136.75 ± 330.00,720.29 ± 19.65,588.02 ± 464.55,3285.68 ± 21681.75
1,DA_fee,2959.49 ± 394.58,214.61 ± 17.87,-8133.80 ± 334.45,719.71 ± 18.90,502.71 ± 491.14,3400.22 ± 22186.07
2,FX_fee,2955.61 ± 394.48,214.51 ± 17.71,-8110.19 ± 334.95,719.97 ± 19.43,482.19 ± 494.82,3461.64 ± 22166.52
3,OB_fee,2181.64 ± 344.08,159.28 ± 17.44,-8154.65 ± 283.56,720.41 ± 17.78,1574.56 ± 417.85,2406.38 ± 22150.63


In [16]:
# Bear Market

show_stats_for_data_alias("bear_market")

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,1155.49 ± 166.46,133.34 ± 14.50,-8094.01 ± 292.10,720.22 ± 19.39,2670.72 ± 292.77,-639.54 ± 10923.09
1,DA_fee,1167.34 ± 164.28,136.92 ± 14.65,-8074.85 ± 294.90,718.49 ± 18.78,2630.45 ± 309.95,-570.27 ± 11279.39
2,FX_fee,1171.16 ± 166.83,136.91 ± 15.45,-8083.02 ± 286.03,718.77 ± 19.10,2633.46 ± 299.82,-518.62 ± 11327.82
3,OB_fee,858.84 ± 157.26,99.09 ± 14.30,-8070.63 ± 293.18,717.85 ± 17.94,3127.09 ± 303.02,-992.18 ± 11285.06
